# My Adverserial Conversation
J. McInerney, 26 May 2025
I am taking some cells from the Week2, Day 1 notebook and modifying them so I can have an adverserial conversation between OpenAI and a local LLM (gemma3:12b).  First I will just reimplement what Ed did in the Week2, Day 1 notebook.  Then I will try a deeper conversation.

In [0]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
#import anthropic
from IPython.display import Markdown, display, update_display

In [0]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    


In [0]:
# Connect to OpenAI, Anthropic
openai = OpenAI()

## An adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [0]:
# Let's make a conversation between GPT-4o-mini and Gemma3:12b
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
local_model = 'gemma3:12b'

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

local_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
local_messages = ["Hi"]

In [0]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, local in zip(gpt_messages, local_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": local})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [0]:
call_gpt()

In [0]:
basellm = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
def call_local():
    messages = []
    for gpt, local_message in zip(gpt_messages, local_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": local_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    
    completion = basellm.chat.completions.create(
        model=local_model,
        messages=messages
    )
    
    return completion.choices[0].message.content

In [0]:
call_local()

In [0]:
call_gpt()

In [0]:
gpt_messages = ["Hi there"]
local_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"local:\n{local_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    local_next = call_local()
    print(f"local:\n{local_next}\n")
    local_messages.append(local_next)

## Let's try a more thoughful conversation
The two chatbots will engage in a friendly discussion on whether the US should have entered World War I in 1917.  They are both open minded so they can learn from each other.

In [0]:
# Let's make a conversation between GPT-4o-mini and Gemma3:12b
# We're using cheap versions of models so the costs will be minimal

gpt_system = "You are a chatbot who believes it was a mistake for the US to enter World War I; \
you are open to other arguments, but you feel the evidence suggests the world would have been \
better off if the US had stayed isolationalist.   You consider counter arguments but also express \
your own arguments."

local_system = "You are a chatbot who believes the US made the right decision entering World War I in \
1917.  Overall, the world is a better place for it.  You are open minded but believe the evidence \
supports this view.  You consider counter arguments but also express your own arguments."

gpt_messages = ["It was such a mistake for the US to enter WWI"]
local_messages = ["Why do you say that?"]

In [0]:
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"local:\n{local_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    local_next = call_local()
    print(f"local:\n{local_next}\n")
    local_messages.append(local_next)

## Conclusion
I am amazed at how insightful this conversation was.  Not only did they explore all the pros and cons, they began applying those lessons to current day foreign policy.  This looks like a very good way to explore a topic. 